In [1]:
from clickhouse_connect.datatypes.format import set_read_format
import clickhouse_connect
import os
import panel as pn
import polars as pl

pn.extension("plotly", template="material", sizing_mode="stretch_width")
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

# Clickhouse Format Settings - https://clickhouse.com/docs/en/integrations/python#read-formats

# Return both IPv6 and IPv4 values as strings
set_read_format("IPv*", "string")

# Return binary as string
set_read_format("FixedString", "string")

# sets large ints to floats so that there are no large int overflow errors when converting to polars dataframe
set_read_format("Int*", "float")

# Create ClickHouse client
client = clickhouse_connect.get_client(
    host=os.environ.get("HOST"),
    username=os.environ.get("USERNAME"),
    password=os.environ.get("PASSWORD"),
    secure=True,
)

In [2]:
# labeled blobs - https://dune.com/queries/3521610 , uses "from" address to classify blobs.
# we start by filtering the mempool for base transactions. Note the dune dashboard addresses are lowercase, but clickhouse addresses are properly checksummed
query = """
SELECT
    # MIN(event_date_time) AS earliest_event_date_time, -- not sure if I want this for now.
    event_date_time,
    type,
    blob_sidecars_size,
    blob_sidecars_empty_size,
    hash,
    to,
    from,
    blob_hashes,
    nonce,
    meta_network_name,
    meta_client_implementation,
    meta_client_geo_continent_code,
    length(blob_hashes) as blob_hashes_length,
    ROUND(100 - (blob_sidecars_empty_size / blob_sidecars_size) * 100, 2) AS fill_percentage,
    blob_gas,
    blob_gas_fee_cap,
    gas_price,
    gas_tip_cap,
    gas_fee_cap
FROM mempool_transaction
WHERE event_date_time > NOW() - INTERVAL '21 DAYS'
AND meta_network_name = 'mainnet'
AND from = '0x5050F69a9786F081509234F1a7F4684b5E5b76C9'
# GROUP BY hash, type, blob_sidecars_size, blob_sidecars_empty_size, blob_hashes, nonce, meta_network_name, blob_hashes_length, fill_percentage, to, from
"""

mempool_df = client.query_df(query)

mempool_df_pl = pl.from_pandas(mempool_df) # 6.8gb

In [3]:
mempool_df_pl.shape

(38782, 19)

In [4]:
mempool_df_pl.head(5)

event_date_time,type,blob_sidecars_size,blob_sidecars_empty_size,hash,to,from,blob_hashes,nonce,meta_network_name,meta_client_implementation,meta_client_geo_continent_code,blob_hashes_length,fill_percentage,blob_gas,blob_gas_fee_cap,gas_price,gas_tip_cap,gas_fee_cap
datetime[ms],u8,u32,u32,str,str,str,list[str],u64,str,str,str,u64,f64,u64,i64,i64,i64,i64
2024-03-21 09:24:40.650,3,786432,3287,"""0x84e7db4c7e81d81f1138b07de94ca9ae66abfc9347a9c02af3ea3f70d167d510""","""0xFf00000000000000000000000000000000008453""","""0x5050F69a9786F081509234F1a7F4684b5E5b76C9""","[""0x01e44db527ebb02896258977dba5ca21cecddd286ba6c62d7b92e47deaa93557"", ""0x01408a3761bf175c87f9ab134e0a5e0379887615823cfceff3ab46e2a74266c3"", … ""0x01e0d964e92b0b763f0daff5698ef0ae0f967ba6b6c8a818978e5ac6150630e5""]",375447,"""mainnet""","""Xatu""","""EU""",6,99.58,786432,1000000000,52811744128,2000000000,52811744128
2024-03-21 09:24:40.934,3,786432,3287,"""0x84e7db4c7e81d81f1138b07de94ca9ae66abfc9347a9c02af3ea3f70d167d510""","""0xFf00000000000000000000000000000000008453""","""0x5050F69a9786F081509234F1a7F4684b5E5b76C9""","[""0x01e44db527ebb02896258977dba5ca21cecddd286ba6c62d7b92e47deaa93557"", ""0x01408a3761bf175c87f9ab134e0a5e0379887615823cfceff3ab46e2a74266c3"", … ""0x01e0d964e92b0b763f0daff5698ef0ae0f967ba6b6c8a818978e5ac6150630e5""]",375447,"""mainnet""","""Xatu""","""AF""",6,99.58,786432,1000000000,52811744128,2000000000,52811744128
2024-03-21 09:24:41.608,3,786432,3287,"""0x84e7db4c7e81d81f1138b07de94ca9ae66abfc9347a9c02af3ea3f70d167d510""","""0xFf00000000000000000000000000000000008453""","""0x5050F69a9786F081509234F1a7F4684b5E5b76C9""","[""0x01e44db527ebb02896258977dba5ca21cecddd286ba6c62d7b92e47deaa93557"", ""0x01408a3761bf175c87f9ab134e0a5e0379887615823cfceff3ab46e2a74266c3"", … ""0x01e0d964e92b0b763f0daff5698ef0ae0f967ba6b6c8a818978e5ac6150630e5""]",375447,"""mainnet""","""Xatu""","""SA""",6,99.58,786432,1000000000,52811744128,2000000000,52811744128
2024-03-22 09:05:45.781,3,786432,3710,"""0x2e5a22251bacbd6eac354fcbcf1a4c2f64301bd8882e01280979d8ae6fe836db""","""0xFf00000000000000000000000000000000008453""","""0x5050F69a9786F081509234F1a7F4684b5E5b76C9""","[""0x017244117c73fff4ae658dec00c88e884c0f968e4b23754313e1962cd88d79a9"", ""0x0110c84a242674c07c32b29e16413d33bad4c9bcb23b3435f1c6724a98c6fdaa"", … ""0x01ba41d068ab4bb8e5d80c55e18c190dddf73baed846a60f57603b3338312c11""]",375749,"""mainnet""","""Xatu""","""SA""",6,99.53,786432,1000000000,53712465436,2000000000,53712465436
2024-03-23 21:19:59.823,3,786432,1437,"""0x51471c54d7adbf93780fa6f8a05fed38fe33a8148a0c2c86a5cd5f1b43a7ab14""","""0xFf00000000000000000000000000000000008453""","""0x5050F69a9786F081509234F1a7F4684b5E5b76C9""","[""0x014a380194cf51764b1deffe5633ea31004722fe9a276b0eeb8bbabece49db2e"", ""0x0126258632c548caf2ad7f7a49463c9a06208b524caaf78baa798d03012a4f60"", … ""0x017748fff3283b6c734747432686981cb281da2585b296a22c8c443e84a96676""]",376239,"""mainnet""","""Xatu""","""OC""",6,99.82,786432,1000000000,32368395786,2000000000,32368395786


In [5]:
# TODO - mark base rollup blob activity, determine patterns, gas bidding amounts problems, etc
mempool_df_pl.write_parquet('base_blob_analysis_3w.parquet')